In [1]:
import pandas as pd

In [2]:
movie_reviews = pd.read_csv('IMDB Dataset.csv').filter(['review'])

In [3]:
def remove_duplicates(list_):
    collect = set()
    for item in list_:
        if item not in collect:
            collect.add(item)
    collect = list(collect)
    return collect

In [4]:
reviews = list(movie_reviews['review'])
reviews = remove_duplicates(reviews)

In [5]:
len(reviews)

49582

In [8]:
movie_synopses = pd.read_csv(r'movies_metadata.csv', low_memory=False, dtype='string').filter(['overview'])
synopses = list(movie_synopses['overview'])
synopses = remove_duplicates(synopses)

In [9]:
reviews = list(filter(lambda x: not pd.isna(x), reviews))
synopses = list(filter(lambda x: not pd.isna(x), synopses))

In [10]:
print(len(reviews), len(synopses))

49582 44307


In [11]:
# down sample reviews
reviews = reviews[:len(synopses)]

In [12]:
len(reviews) == len(synopses), len(reviews), len(synopses)

(True, 44307, 44307)

In [13]:
data = reviews + synopses
len(data), len(data)/2, len(synopses), len(reviews)

(88614, 44307.0, 44307, 44307)

In [14]:
# reduce no. characters to 240 (like Twitter, and to speed up training)
data = [text[:240] for text in data]
len(data)

88614

##### Duplicates occur again since some reviews/synopses might be the same for the first 240 characters

In [15]:
size = len(data)
labeled_reviews = [(data[i], 'Subjective') for i in range(int(size/2))]
labeled_synopses = [(data[i], 'Objective') for i in range(int(size/2), size)]
# labeled_data.append( (data[i], 'Objective') for i in range(int(size/2), size) )

In [16]:
len(labeled_reviews), len(labeled_synopses), len(labeled_reviews) + len(labeled_synopses)

(44307, 44307, 88614)

In [17]:
labeled_reviews = remove_duplicates(labeled_reviews)
labeled_synopses = remove_duplicates(labeled_synopses)

In [18]:
len(labeled_reviews), len(labeled_synopses), len(labeled_reviews) + len(labeled_synopses)

(44285, 44303, 88588)

In [19]:
from random import shuffle
shuffle(labeled_reviews)
shuffle(labeled_synopses)

In [20]:
labeled_data = labeled_reviews + labeled_synopses

In [22]:
shuffle(labeled_data)

In [26]:
df = pd.DataFrame(labeled_data)

In [27]:
df

,0,1
0,"This film, for an after school special, isn't ...",Subjective
1,Dictated by thin experience (of both life and ...,Subjective
2,"This focuses around the lives of four women, a...",Subjective
3,This owes a great deal to the plot of CAPTAINS...,Subjective
4,"Justin goes home to live with his strict, hard...",Subjective
...,...,...
88583,"A film by Louis Aimé Augustin Le Prince, shot ...",Objective
88584,A group of Irish noblemen kidnap girls in orde...,Objective
88585,"The filmmaker inhaled Andy Goldsworthy's art, ...",Subjective
88586,There are few films that leave me with the fee...,Subjective


In [ ]:
from transformers import AutoTokenizer

In [ ]:
rbs_model = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(rbs_model)

In [ ]:
def tokenize(data_dict):
    return tokenizer(data_dict['text'])